In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
with open('intents.json','r') as f:
    file=json.load(f)
file['intents']

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': ['Hello!',
   'Good to see you again!',
   'Hi there, how can I help?'],
  'context_set': ''},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day',
   'bye'],
  'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
  'context_set': ''},
 {'tag': 'age',
  'patterns': ['how old',
   'how old is tim',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': ['I am 18 years old!', '18 years young!'],
  'context_set': ''},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Tim.',
   "I'm Tim!",
   "I'm Tim aka Tech With Tim."],
  'context_set': ''},
 {'tag': 'shop',
  'patterns': ['Id like to buy something',
   'whats on the menu',
   'what do you reccommend?',
   'could

In [3]:
print(file['intents'])
for intent in file['intents']:
    print(intent['tag'],intent['responses'],intent['patterns'])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'bye'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies are on 

In [4]:
tags=[]
patterns=[]
responses={}
context={}
for intent in file['intents']:
    responses[intent['tag']]=intent['responses']
    for line in intent['patterns']:
        patterns.append(line)
        tags.append(intent['tag'])

In [5]:
print(tags)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'age', 'age', 'age', 'age', 'age', 'name', 'name', 'name', 'shop', 'shop', 'shop', 'shop', 'hours', 'hours', 'hours']


In [6]:
print(responses)

{'greeting': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'goodbye': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'age': ['I am 18 years old!', '18 years young!'], 'name': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'shop': ['We sell chocolate chip cookies for $2!', 'Cookies are on the menu!'], 'hours': ['We are open 7am-4pm Monday-Friday!']}


In [7]:
print(patterns)

['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up', 'cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'bye', 'how old', 'how old is tim', 'what is your age', 'how old are you', 'age?', 'what is your name', 'what should I call you', 'whats your name?', 'Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat', 'when are you guys open', 'what are your hours', 'hours of operation']


In [8]:
df=pd.DataFrame({'patterns':patterns,'tags':tags})

In [9]:
df.head()

,patterns,tags
0,Hi,greeting
1,How are you,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   patterns  27 non-null     object
 1   tags      27 non-null     object
dtypes: object(2)
memory usage: 560.0+ bytes


In [11]:
import string
df['patterns']=df.patterns.apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
df['patterns']=df.patterns.apply(lambda wrd: ''.join(wrd))

In [12]:
df.head()

,patterns,tags
0,hi,greeting
1,how are you,greeting
2,is anyone there,greeting
3,hello,greeting
4,good day,greeting


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=2000)
tokenizer.fit_on_texts(df['patterns'])
train=tokenizer.texts_to_sequences(df['patterns'])

from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train=pad_sequences(train)

from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y_train=le.fit_transform(df['tags'])

In [14]:
len(train)

27

In [15]:
input_shape=x_train.shape[1]

In [16]:
vocabulary=len(tokenizer.word_index)
vocabulary
output_shape=le.classes_.shape[0]
output_shape

6

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM,Dropout,GlobalMaxPooling1D,Flatten,Input,Embedding,Dense

In [18]:
inpt=Input(shape=(input_shape,))
embedding=Embedding(vocabulary+1,10)(inpt)
rnn=LSTM(10,return_sequences=True)(embedding)
flatten=Flatten()(rnn)
output=Dense(output_shape,activation='softmax')(flatten)
model=Model(inpt,output)

In [19]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6)]               0         
                                                                 
 embedding (Embedding)       (None, 6, 10)             500       
                                                                 
 lstm (LSTM)                 (None, 6, 10)             840       
                                                                 
 flatten (Flatten)           (None, 60)                0         
                                                                 
 dense (Dense)               (None, 6)                 366       
                                                                 
Total params: 1,706
Trainable params: 1,706
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(x_train,y_train,epochs=300)

Epoch 1/300
1/1 [==============================] - 3s 3s/step - loss: 1.7919 - accuracy: 0.1111
Epoch 2/300
1/1 [==============================] - 0s 10ms/step - loss: 1.7905 - accuracy: 0.1852
Epoch 3/300
1/1 [==============================] - 0s 18ms/step - loss: 1.7890 - accuracy: 0.4074
Epoch 4/300
1/1 [==============================] - 0s 15ms/step - loss: 1.7876 - accuracy: 0.4444
Epoch 5/300
1/1 [==============================] - 0s 16ms/step - loss: 1.7861 - accuracy: 0.4444
Epoch 6/300
1/1 [==============================] - 0s 18ms/step - loss: 1.7847 - accuracy: 0.3704
Epoch 7/300
1/1 [==============================] - 0s 19ms/step - loss: 1.7832 - accuracy: 0.4074
Epoch 8/300
1/1 [==============================] - 0s 15ms/step - loss: 1.7817 - accuracy: 0.3333
Epoch 9/300
1/1 [==============================] - 0s 9ms/step - loss: 1.7802 - accuracy: 0.3333
Epoch 10/300
1/1 [==============================] - 0s 9ms/step - loss: 1.7787 - accuracy: 0.3333
Epoch 11/300
1/1 [=====

1/1 [==============================] - 0s 8ms/step - loss: 1.4123 - accuracy: 0.5185
Epoch 84/300
1/1 [==============================] - 0s 18ms/step - loss: 1.4035 - accuracy: 0.5185
Epoch 85/300
1/1 [==============================] - 0s 12ms/step - loss: 1.3947 - accuracy: 0.5185
Epoch 86/300
1/1 [==============================] - 0s 18ms/step - loss: 1.3859 - accuracy: 0.5185
Epoch 87/300
1/1 [==============================] - 0s 13ms/step - loss: 1.3772 - accuracy: 0.5185
Epoch 88/300
1/1 [==============================] - 0s 13ms/step - loss: 1.3685 - accuracy: 0.5556
Epoch 89/300
1/1 [==============================] - 0s 12ms/step - loss: 1.3599 - accuracy: 0.5556
Epoch 90/300
1/1 [==============================] - 0s 18ms/step - loss: 1.3513 - accuracy: 0.5556
Epoch 91/300
1/1 [==============================] - 0s 14ms/step - loss: 1.3428 - accuracy: 0.5556
Epoch 92/300
1/1 [==============================] - 0s 14ms/step - loss: 1.3344 - accuracy: 0.5556
Epoch 93/300
1/1 [======

1/1 [==============================] - 0s 13ms/step - loss: 0.8665 - accuracy: 0.7407
Epoch 166/300
1/1 [==============================] - 0s 6ms/step - loss: 0.8608 - accuracy: 0.7407
Epoch 167/300
1/1 [==============================] - 0s 12ms/step - loss: 0.8551 - accuracy: 0.7407
Epoch 168/300
1/1 [==============================] - 0s 16ms/step - loss: 0.8494 - accuracy: 0.7407
Epoch 169/300
1/1 [==============================] - 0s 18ms/step - loss: 0.8437 - accuracy: 0.7407
Epoch 170/300
1/1 [==============================] - 0s 16ms/step - loss: 0.8380 - accuracy: 0.7407
Epoch 171/300
1/1 [==============================] - 0s 0s/step - loss: 0.8323 - accuracy: 0.7778
Epoch 172/300
1/1 [==============================] - 0s 17ms/step - loss: 0.8267 - accuracy: 0.7778
Epoch 173/300
1/1 [==============================] - 0s 17ms/step - loss: 0.8210 - accuracy: 0.7778
Epoch 174/300
1/1 [==============================] - 0s 17ms/step - loss: 0.8154 - accuracy: 0.7778
Epoch 175/300
1/1

1/1 [==============================] - 0s 9ms/step - loss: 0.4403 - accuracy: 1.0000
Epoch 248/300
1/1 [==============================] - 0s 14ms/step - loss: 0.4358 - accuracy: 1.0000
Epoch 249/300
1/1 [==============================] - 0s 7ms/step - loss: 0.4313 - accuracy: 1.0000
Epoch 250/300
1/1 [==============================] - 0s 0s/step - loss: 0.4268 - accuracy: 1.0000
Epoch 251/300
1/1 [==============================] - 0s 5ms/step - loss: 0.4224 - accuracy: 1.0000
Epoch 252/300
1/1 [==============================] - 0s 3ms/step - loss: 0.4180 - accuracy: 1.0000
Epoch 253/300
1/1 [==============================] - 0s 19ms/step - loss: 0.4136 - accuracy: 1.0000
Epoch 254/300
1/1 [==============================] - 0s 16ms/step - loss: 0.4093 - accuracy: 1.0000
Epoch 255/300
1/1 [==============================] - 0s 19ms/step - loss: 0.4050 - accuracy: 1.0000
Epoch 256/300
1/1 [==============================] - 0s 15ms/step - loss: 0.4007 - accuracy: 1.0000
Epoch 257/300
1/1 [=

In [22]:
import random

while True:
    text_p=[]
    prediction_input=input('You :- ')
    
#     removing punt and converting to lower_case
    prediction_input=[letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input=''.join(prediction_input)
    text_p.append(prediction_input)
    
#     tokenizing and padding
    prediction_input=tokenizer.texts_to_sequences(text_p)
    prediction_input=np.array(prediction_input).reshape(-1)
    prediction_input=pad_sequences([prediction_input],input_shape)
    
#     getting output from model

    output=model.predict(prediction_input)
    output=output.argmax()
    
#     finding the right tag and predicting 
    response_tag=le.inverse_transform([output])[0]
    print('Intelligent Bot :- ',random.choice(responses[response_tag]))
    if response_tag=='goodbye':
        break

You :- hi
1/1 [==============================] - 1s 531ms/step
Intelligent Bot :-  Hello!
You :- hi
1/1 [==============================] - 0s 31ms/step
Intelligent Bot :-  Hello!
You :- ji
1/1 [==============================] - 0s 31ms/step
Intelligent Bot :-  Goodbye!
